In [10]:
import os
import pickle
import numpy as np
from mayavi import mlab
from plyfile import PlyData
import networkx as nx
from scipy import sparse, spatial
import datetime

In [13]:
# import pcst_fast

# TODO: Try a larger cube, apply a threshold

'2019-10-23 03:22:21.732751'

In [31]:
def load_skels_from_dir(directory):
    pickle_path = os.path.join(directory, "parsed_graph.pickle")

    if os.path.exists(pickle_path):
        with open(pickle_path, "rb") as fi:
            print("Loading skeleton data from pickle")
            vertices, edges, vertex_properties, G = pickle.load(fi)
            return [vertices, edges, vertex_properties, G]
    else:
        print("Loading skeleton data from directory")
        files = [os.path.join(directory, fi)
                 for fi in os.listdir(directory) if fi.endswith(".ply")]
        skel1 = PlyData.read(files[0])
        vertices = np.array(skel1["vertex"].data.tolist())
        vertex_properties = vertices[:, [0, 1]]
        vertices = vertices[:, 2:]
        edges = np.array(skel1["edge"].data.tolist())
        begin_verts = vertices[edges[:, 0], :]
        end_verts = vertices[edges[:, 1], :]
        vector_components = end_verts - begin_verts

        # Load a list of ply files
        for fi in files[1:]:
            skel = PlyData.read(fi)
            _vertices = np.array(skel["vertex"].data.tolist())
            _vertex_properties = _vertices[:, [0, 1]]
            _vertices = _vertices[:, 2:]
            _edges = np.array(skel["edge"].data.tolist())

            # Increment all the indices in edges so that they match up with the full vertex array
            edges = np.concatenate([edges, (_edges + vertices.shape[0])], axis=0)

            vertices = np.concatenate([vertices, _vertices], axis=0)
            vertex_properties = np.concatenate([vertex_properties, _vertex_properties], axis=0)

        G = nx.Graph()
        G.add_edges_from(edges)

        with open(pickle_path, "wb") as fi:
            print("Writing skeleton data as pickle")
            pickle.dump([vertices, edges, vertex_properties, G], fi)

    return [vertices, edges, vertex_properties, G]


def make_filtered_graph(_vertices, _vertex_properties, _edges, threshold):
    vertex_filter = _vertex_properties[:, 1] > threshold
    good_indices = np.arange(_vertices.shape[0])[vertex_filter]
    edge_filter = np.logical_and(np.isin(_edges[:, 0], good_indices),
                                 np.isin(_edges[:, 1], good_indices))
    filtered_verts = _vertices[vertex_filter, :]
    filtered_edges = _edges[edge_filter, :]
    filtered_begin_verts = _vertices[filtered_edges[:, 0], :]
    filtered_end_verts = _vertices[filtered_edges[:, 1], :]
    filtered_vector_components = filtered_end_verts - filtered_begin_verts
    filtered_distances = np.sqrt(np.sum(np.square(filtered_end_verts - filtered_begin_verts), axis=1))
    mlab.points3d(filtered_verts[:, 0], filtered_verts[:, 1], filtered_verts[:, 2],
                  np.sqrt(_vertex_properties[vertex_filter, 1]),
                  opacity=0.1)

    mlab.quiver3d(filtered_begin_verts[:, 0], filtered_begin_verts[:, 1], filtered_begin_verts[:, 2],
                  filtered_vector_components[:, 0], filtered_vector_components[:, 1],
                  filtered_vector_components[:, 2], scalars=filtered_distances, scale_mode="scalar", scale_factor=1,
                  mode="2ddash", opacity=1.)


def plot_subgraph_old(vertices, edges, vertex_properties, subgraph_no):
    _, subgraph_edges = get_subgraph(edges, subgraph_no)
    begin_verts = vertices[subgraph_edges[:, 0], :]
    end_verts = vertices[subgraph_edges[:, 1], :]
    vector_components = end_verts - begin_verts
    distances = np.sqrt(np.sum(np.square(end_verts - begin_verts), axis=1))
    vertex_mask = np.isin(np.arange(vertices.shape[0]), subgraph_edges)

    mlab.points3d(vertices[vertex_mask, 0], vertices[vertex_mask, 1], vertices[vertex_mask, 2],
                  vertex_properties[vertex_mask, 1], scale_mode='scalar', scale_factor=1, opacity=0.1)
    mlab.quiver3d(begin_verts[:, 0], begin_verts[:, 1], begin_verts[:, 2], vector_components[:, 0],
                  vector_components[:, 1],
                  vector_components[:, 2], scalars=distances, mode="2ddash", opacity=1., scale_mode="scalar",
                  scale_factor=1, color=(1, 1, 1))


def plot_subgraph(subgraph, vertices, points_kwargs={}, quiver_kwargs={}):
    subgraph_edges = np.array(subgraph.edges)
    begin_verts = vertices[subgraph_edges[:, 0], :]
    end_verts = vertices[subgraph_edges[:, 1], :]
    vector_components = end_verts - begin_verts
    distances = np.sqrt(np.sum(np.square(end_verts - begin_verts), axis=1))
    vertex_mask = np.unique(subgraph_edges.reshape((-1, 1)))

    mlab.points3d(vertices[vertex_mask, 0], vertices[vertex_mask, 1], vertices[vertex_mask, 2],
                  scale_mode='scalar', scale_factor=1, opacity=0.1, **points_kwargs)
    mlab.quiver3d(begin_verts[:, 0], begin_verts[:, 1], begin_verts[:, 2], vector_components[:, 0],
                  vector_components[:, 1],
                  vector_components[:, 2], scalars=distances, mode="2ddash", opacity=1., scale_mode="scalar",
                  scale_factor=1, **quiver_kwargs)


def get_subgraph(edges, subgraph_no):
    G = nx.Graph()
    G.add_edges_from(edges)
    subgraphs = list(sorted(nx.connected_components(G), key=len, reverse=True))
    root_subgraph = G.subgraph(subgraphs[subgraph_no])
    subgraph_edges = np.array(root_subgraph.edges)
    return root_subgraph, subgraph_edges


def plot_skeleton(skel):
    vertices, edges, vertex_properties = get_skel_data(skel)

    G = nx.Graph()
    G.add_edges_from(edges)

    plot_edges(edges, vertices)

    return plot_nodes(vertices, vertex_properties)


def plot_nodes(vertices, vertex_properties, **kwargs):
    plot_args = {"opacity": 0.05, "scale_mode": 'scalar', "scale_factor": 1, }
    plot_args.update(kwargs)
    return mlab.points3d(vertices[:, 0], vertices[:, 1], vertices[:, 2], vertex_properties[:, 1], **kwargs)


def get_skel_data(skel):
    vertices = np.array(skel["vertex"].data.tolist())
    vertex_properties = vertices[:, [0, 1]]
    vertices = vertices[:, 2:]
    edges = np.array(skel["edge"].data.tolist())
    return vertices, edges, vertex_properties


def plot_edges(edges, vertices, **kwargs):
    plot_args = {"mode": "2ddash", "opacity": 1., "scale_mode": "scalar",
                 "scale_factor": 1.}
    plot_args.update(kwargs)
    begin_verts = vertices[edges[:, 0], :]
    end_verts = vertices[edges[:, 1], :]
    vector_components = end_verts - begin_verts
    distances = np.sqrt(np.sum(np.square(end_verts - begin_verts), axis=1))
    return mlab.quiver3d(begin_verts[:, 0], begin_verts[:, 1], begin_verts[:, 2], vector_components[:, 0],
                         vector_components[:, 1],
                         vector_components[:, 2], scalars=distances, **plot_args)


# plot_subgraph(vertices, edges, vertex_properties, 0)

# skel1 = PlyData.read("v2highres_skel.ply")
# vertices, edges, vertex_properties = get_skel_data(skel1)
# # plot_skeleton(skel1)
# plot_edges(edges, vertices)
# plot_edges(cycle1, vertices, color=(1, 0, 0), line_width=10.0)

def get_adjacent_points(center, vertices, side_length):
    half = side_length / 2.
    cx, cy, cz = center
    vx = vertices[:, 0]
    vy = vertices[:, 1]
    vz = vertices[:, 2]
    selected = vertices[np.logical_and.reduce(
        (vx >= (cx - half), vx <= (cx + half),
         vy >= (cy - half), vy <= (cy + half),
         vz >= (cz - half), vz <= (cz + half))), :]
    return selected


def get_subgraph_vector(subgraph, vertices, mean_root_vert, get_distal_leaf=False):
    degrees = np.array(subgraph.degree())
    leaf_indices = degrees[degrees[:, 1] == np.min(degrees[:, 1]), 0]
    leaf_verts = vertices[leaf_indices, :]
    #     try:
    distal_leaf = leaf_verts[np.argmax(spatial.distance.cdist(mean_root_vert, leaf_verts)), :]
    #     except Exception as e:
    #         print("Degrees: {}".format(degrees))
    #         print("Leaf verts:\n{}".format(leaf_verts))
    #         print("Leaf indices:\n{}".format(leaf_indices))
    #         raise e
    root_vector = np.mean(vertices[subgraph.nodes, :] - distal_leaf, axis=0)
    norm = np.linalg.norm(root_vector)
    if norm != 0:
        root_vector = root_vector / norm
    if not get_distal_leaf:
        return root_vector.reshape([1, -1])
    else:
        return root_vector.reshape([1, -1]), distal_leaf.reshape([1, -1])

ELLIPTICAL_DISTANCE_PLOT = False

def get_elliptical_distance(s_verts, d_verts, alpha, vector):
    # s_verts: m x 3
    # d_verts: o x 3
    # f(a,b): o x 1 x m x 3
    d_verts_br = d_verts[:, None, None, :]

    dxdxdz = d_verts_br - s_verts
    if ELLIPTICAL_DISTANCE_PLOT:
        vectors = dxdxdz[0,0,:,:]
        norms = np.linalg.norm(vectors, axis=1)
        print(s_verts)
        print(vectors)
        print(norms)

        dest_vertex = d_verts[0, :].reshape([1, -1])
        mlab.points3d(dest_vertex[:, 0], dest_vertex[:, 1], dest_vertex[:,2], color=(0,1,0))

        mlab.quiver3d(s_verts[:, 0], s_verts[:, 1], s_verts[:, 2], vectors[:, 0],
                         vectors[:, 1],
                         vectors[:, 2], scalars=norms, scale_mode="scalar", scale_factor=1, line_width=5)

    dx = dxdxdz[:, :, :, 0]
    dy = dxdxdz[:, :, :, 1]
    dz = dxdxdz[:, :, :, 2]
    sq_distance = dx * dx + dy * dy + dz * dz

    final_distance = np.sqrt(sq_distance -
                             alpha * np.square((dx * vector[:, 0] + dy * vector[:, 1] + dz * vector[:, 2])))
                             
    final_distance = np.transpose(final_distance.reshape(d_verts.shape[0], s_verts.shape[0]))
    return final_distance


def get_elliptical_distance_bi(s_verts, d_verts, alpha, s_vectors, beta, d_vectors):
    # s_verts: m x 3
    # d_verts: o x 3
    # f(a,b): o x 1 x m x 3
    d_verts_br = d_verts[:, None, None, :]

    dxdxdz = d_verts_br - s_verts
    if ELLIPTICAL_DISTANCE_PLOT:
        vectors = dxdxdz[0,0,:,:]
        norms = np.linalg.norm(vectors, axis=1)
        print(s_verts)
        print(vectors)
        print(norms)

        dest_vertex = d_verts[0, :].reshape([1, -1])
        mlab.points3d(dest_vertex[:, 0], dest_vertex[:, 1], dest_vertex[:,2], color=(0,1,0))

        mlab.quiver3d(s_verts[:, 0], s_verts[:, 1], s_verts[:, 2], vectors[:, 0],
                         vectors[:, 1],
                         vectors[:, 2], scalars=norms, scale_mode="scalar", scale_factor=1, line_width=5)

    dx = dxdxdz[:, :, :, 0]
    dy = dxdxdz[:, :, :, 1]
    dz = dxdxdz[:, :, :, 2]
    sq_distance = dx * dx + dy * dy + dz * dz

    final_distance = np.sqrt(sq_distance -
                             alpha * np.square((dx * s_vectors[:, 0] + dy * s_vectors[:, 1] + dz * s_vectors[:, 2])) -
                             beta * np.square((dx * d_vectors[:, 0] + dy * d_vectors[:, 1] + dz * d_vectors[:, 2])))
    final_distance = np.transpose(final_distance.reshape(d_verts.shape[0], s_verts.shape[0]))
    return final_distance


def get_best_edge_between_subgraphs(subgraph_s, subgraph_d, s_vector, verts_and_idx):
    # Get nodes with smallest distance between them and the distance
    s_degrees = np.array(subgraph_s.degree())
    s_leaf_indices = s_degrees[s_degrees[:, 1] == np.min(s_degrees[:, 1]), 0]
    s_verts = verts_and_idx[s_leaf_indices, :]
    d_verts = verts_and_idx[subgraph_d.nodes, :]

    distances = get_elliptical_distance(s_verts[:, :3], d_verts[:, :3], 0.8, s_vector)
    min_distance_idx = np.unravel_index(distances.argmin(), distances.shape)
    min_distance_edge = (s_verts[min_distance_idx[0], 3], d_verts[min_distance_idx[1], 3], distances[min_distance_idx])

    return min_distance_edge

def get_graph_data(directory):
    vertices, edges, vertex_properties, G = load_skels_from_dir(directory)
    verts_and_idx = np.hstack([vertices, np.arange(len(vertices)).reshape(-1, 1)])
    subgraphs = list(sorted(nx.connected_components(G), key=len, reverse=True))
    nx_subgraphs = [G.subgraph(subgraph) for subgraph in subgraphs]
    root_subgraph = nx_subgraphs[0]
    root_indices = root_subgraph.nodes
    root_verts = vertices[root_indices, :]
    root_mean = np.mean(root_verts, axis=0).reshape((1, 3))
    subgraph_vectors = np.array([get_subgraph_vector(subgraph, vertices, root_mean) for subgraph in nx_subgraphs])
    return G, edges, root_indices, root_subgraph, subgraph_vectors, subgraphs, nx_subgraphs, vertices, verts_and_idx

In [32]:
DIRECTORY = "brady129"
MAX_DISTANCE = 30
MIN_NEIGHBORS = 0
BIDIRECTIONAL_DISTANCE = False

G, edges, root_indices, root_subgraph, subgraph_vectors, subgraphs, nx_subgraphs, vertices, verts_and_idx = \
        get_graph_data(DIRECTORY)

Loading skeleton data from pickle


In [7]:
pickle_name = "distance_matrix_{}_alpha_0.8.pickle".format(DIRECTORY)

In [8]:
os.unlink(pickle_name)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'distance_matrix_brady129_alpha_0.8.pickle'

In [19]:
vertices.shape[0]**2

172254881296

In [28]:
num_subgraphs = len(subgraphs)
if not os.path.exists(pickle_name):
    subgraph_node_edges = np.zeros((len(subgraphs), len(subgraphs), 3))

    # Find shortest path between subgraphs
    for i, subgraph in enumerate(subgraphs):
        # Only evaluate lower triangular, diagonal is 0
        if i == 0:
            print("{}: Starting".format(str(datetime.datetime.now())))
            continue
        s_vector = subgraph_vectors[i]
        _subgraph = nx_subgraphs[i]
        best_edges = np.array(
            [get_best_edge_between_subgraphs(_subgraph, subgraph, s_vector, verts_and_idx) 
             
             for subgraph in nx_subgraphs[:i]])
        subgraph_node_edges[i, :i, :] = best_edges
        if i % 10 == 0:
            print("{}: Finished {} subgraphs".format(str(datetime.datetime.now()), i))

    with open(pickle_name, "wb") as pickle_fi:
        pickle.dump(subgraph_node_edges, pickle_fi)
else:
    with open(pickle_name, "rb") as pickle_fi:
        subgraph_node_edges = pickle.load(pickle_fi)

2019-10-23 03:48:49.689366: Starting
2019-10-23 03:48:55.318829: Finished 10 subgraphs
2019-10-23 03:49:02.612027: Finished 20 subgraphs
2019-10-23 03:49:11.552630: Finished 30 subgraphs
2019-10-23 03:49:20.885197: Finished 40 subgraphs
2019-10-23 03:49:32.101196: Finished 50 subgraphs
2019-10-23 03:49:43.877387: Finished 60 subgraphs
2019-10-23 03:49:56.281374: Finished 70 subgraphs
2019-10-23 03:50:09.361021: Finished 80 subgraphs
2019-10-23 03:50:23.711608: Finished 90 subgraphs
2019-10-23 03:50:37.341011: Finished 100 subgraphs
2019-10-23 03:50:51.174715: Finished 110 subgraphs
2019-10-23 03:51:07.552910: Finished 120 subgraphs
2019-10-23 03:51:22.475123: Finished 130 subgraphs
2019-10-23 03:51:38.628067: Finished 140 subgraphs
2019-10-23 03:51:54.002296: Finished 150 subgraphs
2019-10-23 03:52:08.436265: Finished 160 subgraphs
2019-10-23 03:52:23.740358: Finished 170 subgraphs
2019-10-23 03:52:40.152500: Finished 180 subgraphs
2019-10-23 03:52:54.879016: Finished 190 subgraphs
201

2019-10-23 04:33:12.702431: Finished 1600 subgraphs
2019-10-23 04:33:27.706302: Finished 1610 subgraphs
2019-10-23 04:33:42.288319: Finished 1620 subgraphs
2019-10-23 04:33:57.263291: Finished 1630 subgraphs
2019-10-23 04:34:12.512504: Finished 1640 subgraphs
2019-10-23 04:34:26.876098: Finished 1650 subgraphs
2019-10-23 04:34:41.270609: Finished 1660 subgraphs
2019-10-23 04:34:56.461990: Finished 1670 subgraphs
2019-10-23 04:35:10.714884: Finished 1680 subgraphs
2019-10-23 04:35:24.749353: Finished 1690 subgraphs
2019-10-23 04:35:38.700026: Finished 1700 subgraphs
2019-10-23 04:35:52.763448: Finished 1710 subgraphs
2019-10-23 04:36:06.939550: Finished 1720 subgraphs
2019-10-23 04:36:21.184429: Finished 1730 subgraphs
2019-10-23 04:36:35.768459: Finished 1740 subgraphs
2019-10-23 04:36:49.946549: Finished 1750 subgraphs
2019-10-23 04:37:03.964070: Finished 1760 subgraphs
2019-10-23 04:37:18.035418: Finished 1770 subgraphs
2019-10-23 04:37:37.477525: Finished 1780 subgraphs
2019-10-23 0

2019-10-23 05:09:15.841623: Finished 3180 subgraphs
2019-10-23 05:09:27.761749: Finished 3190 subgraphs
2019-10-23 05:09:39.587155: Finished 3200 subgraphs
2019-10-23 05:09:51.656884: Finished 3210 subgraphs
2019-10-23 05:10:03.497223: Finished 3220 subgraphs
2019-10-23 05:10:15.022430: Finished 3230 subgraphs
2019-10-23 05:10:26.387043: Finished 3240 subgraphs
2019-10-23 05:10:38.013930: Finished 3250 subgraphs
2019-10-23 05:10:49.369590: Finished 3260 subgraphs
2019-10-23 05:11:00.812970: Finished 3270 subgraphs
2019-10-23 05:11:12.449877: Finished 3280 subgraphs
2019-10-23 05:11:24.194045: Finished 3290 subgraphs
2019-10-23 05:11:35.909718: Finished 3300 subgraphs
2019-10-23 05:11:47.301258: Finished 3310 subgraphs
2019-10-23 05:11:58.935152: Finished 3320 subgraphs
2019-10-23 05:12:10.429392: Finished 3330 subgraphs
2019-10-23 05:12:22.011449: Finished 3340 subgraphs
2019-10-23 05:12:33.532643: Finished 3350 subgraphs
2019-10-23 05:12:45.124648: Finished 3360 subgraphs
2019-10-23 0

2019-10-23 05:38:00.944291: Finished 4760 subgraphs
2019-10-23 05:38:10.476828: Finished 4770 subgraphs
2019-10-23 05:38:19.598820: Finished 4780 subgraphs
2019-10-23 05:38:28.929870: Finished 4790 subgraphs
2019-10-23 05:38:38.162210: Finished 4800 subgraphs
2019-10-23 05:38:47.324718: Finished 4810 subgraphs
2019-10-23 05:38:56.638781: Finished 4820 subgraphs


IndexError: index 4826 is out of bounds for axis 0 with size 4826

In [29]:
    with open(pickle_name, "wb") as pickle_fi:
        pickle.dump(subgraph_node_edges, pickle_fi)

In [ ]:
"""
2019-10-23 03:25:13.559301: Finished 10 subgraphs
2019-10-23 03:27:07.326356: Finished 20 subgraphs
2019-10-23 03:29:00.122219: Finished 30 subgraphs
2019-10-23 03:30:48.843515: Finished 40 subgraphs
"""

In [ ]:
mirror_ind = numpy.triu_indices(len(subgraphs), -1)
for i in range(3):
    subgraph_node_edges[:,:,i.T()][mirror_ind] = subgraph_node_edges[:,:,i].T()[mirror_ind]

In [ ]:
## # Get nodes with smallest distance between them and the distance
# subgraph_s = nx_subgraphs[4200]
# subgraph_d = nx_subgraphs[4201]
# s_vector = subgraph_vectors[4200]
# s_verts = verts_and_idx[subgraph_s.nodes, :]
# d_verts = verts_and_idx[subgraph_d.nodes, :]

# distances = get_elliptical_distance(s_verts, d_verts, 0.8, s_vector)
# min_distance_idx = np.unravel_index(distances.argmin(), distances.shape)
# min_distance_edge = [s_verts[min_distance_idx[0], 3], d_verts[min_distance_idx[1], 3], distances[min_distance]]

In [ ]:
a = list(range(5))

In [ ]:
a[:0]